# Laboratorium 11: Strojenie hiperparametrow

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

## Poszukiwania reczne

Zestaw danych **Boston Housing**

In [2]:
(X_train, y_train), (X_test, y_test) = keras.datasets.boston_housing.load_data()

funkcja budujaca model

In [3]:
def build_model(n_hidden = 1, n_neurons = 25, optimizer = "sgd",
                learning_rate = 1e-5, momentum=0): 
    
    if optimizer == "sgd": optimizer = keras.optimizers.SGD(learning_rate = learning_rate)
    elif optimizer == "nesterov": optimizer = keras.optimizers.SGD(learning_rate=learning_rate,
                                                                     nesterov = True)
    elif optimizer == "momentum" : optimizer = keras.optimizers.SGD(learning_rate=learning_rate,
                                                                      momentum=momentum)
    elif optimizer == "adam": optimizer = keras.optimizers.Adam(learning_rate = learning_rate)
                                                                     
    model = tf.keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=X_train.shape[1:]))
    for i in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
        
    model.compile(loss="mse", optimizer = optimizer, metrics=["mse", "mae"])
    return model

In [4]:
lr_list = [1e-6, 1e-5, 1e-4]
hl_list = [0, 1, 2, 3]
nn_list = [5, 25, 125]
opt_list = ["sgd", "nesterov", "momentum", "adam"]
mom_list = [0.1, 0.5, 0.9]

In [5]:
import os
root_logdir = os.path.join(os.curdir, "tb_logs")

def get_run_logdir(name, value): 
    import time
    unixtime = int(time.time())
    run_id = "_".join((str(unixtime), name, str(value)))
    return os.path.join(root_logdir, run_id)

In [6]:
def save_to_pickle(name, values):
    import pickle
    name = ".".join((name,"pkl"))
    with open(name,'wb') as f: pickle.dump(values, f)

    #sprawdzenie zawartosci pliku
    with open(name,'rb') as f: print(pickle.load(f))

### eksperymetr 1: krok uczenia (lr): 10−6, 10−5, 10−4

In [7]:
#Przed eksperymentami wyczyść sesję TensorFlow i ustal generatory liczb losowych:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)
lr_pkl = list()

for lr in lr_list:
    tensorboard_cb = keras.callbacks.TensorBoard(get_run_logdir("lr", lr))
    early_stopping = keras.callbacks.EarlyStopping(monitor="loss",
                                                   patience = 10,
                                                   min_delta=1.0,
                                                   verbose=1)
    
    model = build_model(learning_rate=lr)                      
    history = model.fit(X_train, y_train, epochs=100,
                    callbacks=[early_stopping, tensorboard_cb])
    
    lr_pkl.append((lr, history.history["mse"][-1], history.history["mae"][-1]))
save_to_pickle("lr",lr_pkl)

Epoch 1/100


2022-06-04 13:49:40.436355: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


13/13 [==============================] - 0s 2ms/step - loss: 2700.2764 - mse: 2700.2764 - mae: 31.1550
Epoch 2/100
13/13 [==============================] - 0s 3ms/step - loss: 246.2086 - mse: 246.2086 - mae: 11.9515
Epoch 3/100
13/13 [==============================] - 0s 2ms/step - loss: 189.4720 - mse: 189.4720 - mae: 10.5609
Epoch 4/100
13/13 [==============================] - 0s 2ms/step - loss: 175.1508 - mse: 175.1508 - mae: 10.1026
Epoch 5/100
13/13 [==============================] - 0s 2ms/step - loss: 167.5972 - mse: 167.5972 - mae: 9.9639
Epoch 6/100
13/13 [==============================] - 0s 2ms/step - loss: 154.0362 - mse: 154.0362 - mae: 9.4851
Epoch 7/100
13/13 [==============================] - 0s 2ms/step - loss: 147.4924 - mse: 147.4924 - mae: 9.3873
Epoch 8/100
13/13 [==============================] - 0s 2ms/step - loss: 150.3802 - mse: 150.3802 - mae: 9.4128
Epoch 9/100
13/13 [==============================] - 0s 2ms/step - loss: 135.4045 - mse: 135.4045 - mae: 8.771

### eksperymetr 2: liczba warstw ukrytych (hl): od 0 do 3,

In [8]:
#Przed eksperymentami wyczyść sesję TensorFlow i ustal generatory liczb losowych:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)
hr_pkl = list()

for hl in hl_list:
    tensorboard_cb = keras.callbacks.TensorBoard(get_run_logdir("hl", hl))
    early_stopping = keras.callbacks.EarlyStopping(monitor="loss",
                                                   patience = 10,
                                                   min_delta=1.0,
                                                   verbose=1)
    
    model = build_model(n_hidden = hl)                      
    history = model.fit(X_train, y_train, epochs=100,
                    callbacks=[early_stopping, tensorboard_cb])
    
    lr_pkl.append((hl, history.history["mse"][-1], history.history["mae"][-1]))
save_to_pickle("hr",hr_pkl)

Epoch 1/100
13/13 [==============================] - 0s 2ms/step - loss: 24234845961206104064.0000 - mse: 24234845961206104064.0000 - mae: 1486527488.0000
Epoch 2/100
13/13 [==============================] - 0s 2ms/step - loss: inf - mse: inf - mae: 2260918888248115200.0000                                       
Epoch 3/100
13/13 [==============================] - 0s 2ms/step - loss: inf - mse: inf - mae: 4824812911053208938256269312.0000
Epoch 4/100
13/13 [==============================] - 0s 2ms/step - loss: nan - mse: nan - mae: nan                            
Epoch 5/100
13/13 [==============================] - 0s 2ms/step - loss: nan - mse: nan - mae: nan
Epoch 6/100
13/13 [==============================] - 0s 2ms/step - loss: nan - mse: nan - mae: nan
Epoch 7/100
13/13 [==============================] - 0s 2ms/step - loss: nan - mse: nan - mae: nan
Epoch 8/100
13/13 [==============================] - 0s 2ms/step - loss: nan - mse: nan - mae: nan
Epoch 9/100
13/13 [===============

### eksperyment 3: liczba neuronów na warstwę (nn): 5, 25, 125,

In [9]:
#Przed eksperymentami wyczyść sesję TensorFlow i ustal generatory liczb losowych:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)
nn_pkl = list()

for nn in nn_list:
    tensorboard_cb = keras.callbacks.TensorBoard(get_run_logdir("nn", nn))
    early_stopping = keras.callbacks.EarlyStopping(monitor="loss",
                                                   patience = 10,
                                                   min_delta=1.0,
                                                   verbose=1)
    
    model = build_model(n_neurons = nn)                      
    history = model.fit(X_train, y_train, epochs=100,
                    callbacks=[early_stopping, tensorboard_cb])
    
    nn_pkl.append((hl, history.history["mse"][-1], history.history["mae"][-1]))
save_to_pickle("nn", nn_pkl)    

Epoch 1/100
13/13 [==============================] - 0s 2ms/step - loss: 5503.8071 - mse: 5503.8071 - mae: 47.5473
Epoch 2/100
13/13 [==============================] - 0s 3ms/step - loss: 585.7496 - mse: 585.7496 - mae: 22.3859
Epoch 3/100
13/13 [==============================] - 0s 3ms/step - loss: 585.4879 - mse: 585.4879 - mae: 22.3800
Epoch 4/100
13/13 [==============================] - 0s 2ms/step - loss: 585.2284 - mse: 585.2284 - mae: 22.3742
Epoch 5/100
13/13 [==============================] - 0s 2ms/step - loss: 584.9681 - mse: 584.9681 - mae: 22.3684
Epoch 6/100
13/13 [==============================] - 0s 2ms/step - loss: 584.7070 - mse: 584.7070 - mae: 22.3626
Epoch 7/100
13/13 [==============================] - 0s 2ms/step - loss: 584.4481 - mse: 584.4481 - mae: 22.3568
Epoch 8/100
13/13 [==============================] - 0s 2ms/step - loss: 584.1886 - mse: 584.1886 - mae: 22.3510
Epoch 9/100
13/13 [==============================] - 0s 2ms/step - loss: 583.9299 - mse: 583.9

### eksperyment 4: algorytm optymalizacji (opt): wszystkie 4 algorytmy (pęd = 0.5)

In [10]:
#Przed eksperymentami wyczyść sesję TensorFlow i ustal generatory liczb losowych:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)
opt_pkl = list()

for opt in opt_list:
    tensorboard_cb = keras.callbacks.TensorBoard(get_run_logdir("opt", opt))
    early_stopping = keras.callbacks.EarlyStopping(monitor="loss",
                                                   patience = 10,
                                                   min_delta=1.0,
                                                   verbose=1)
    if opt == "momentum": m = 0.5
    else: m=0
    
    model = build_model(optimizer = opt, momentum=m)                      
    history = model.fit(X_train, y_train, epochs=100,
                    callbacks=[early_stopping, tensorboard_cb])
    
    opt_pkl.append((opt, history.history["mse"][-1], history.history["mae"][-1]))
save_to_pickle("opt", opt_pkl)  

Epoch 1/100
13/13 [==============================] - 0s 1ms/step - loss: 83732.3828 - mse: 83732.3828 - mae: 99.9468
Epoch 2/100
13/13 [==============================] - 0s 1ms/step - loss: 122.9807 - mse: 122.9807 - mae: 8.1736
Epoch 3/100
13/13 [==============================] - 0s 2ms/step - loss: 93.0585 - mse: 93.0585 - mae: 7.1528
Epoch 4/100
13/13 [==============================] - 0s 2ms/step - loss: 82.7675 - mse: 82.7675 - mae: 6.6219
Epoch 5/100
13/13 [==============================] - 0s 2ms/step - loss: 76.3989 - mse: 76.3989 - mae: 6.4053
Epoch 6/100
13/13 [==============================] - 0s 2ms/step - loss: 73.2667 - mse: 73.2667 - mae: 6.1773
Epoch 7/100
13/13 [==============================] - 0s 2ms/step - loss: 71.5863 - mse: 71.5863 - mae: 5.9187
Epoch 8/100
13/13 [==============================] - 0s 2ms/step - loss: 68.0666 - mse: 68.0666 - mae: 5.8083
Epoch 9/100
13/13 [==============================] - 0s 2ms/step - loss: 66.3178 - mse: 66.3178 - mae: 5.6422
E

### eksperyment 5: pęd (mom): 0.1, 0.5, 0.9 (dla algorytmu momentum)

In [11]:
#Przed eksperymentami wyczyść sesję TensorFlow i ustal generatory liczb losowych:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)
mom_pkl = list()

for mom in mom_list:
    tensorboard_cb = keras.callbacks.TensorBoard(get_run_logdir("mom", mom))
    early_stopping = keras.callbacks.EarlyStopping(monitor="loss",
                                                   patience = 10,
                                                   min_delta=1.0,
                                                   verbose=1)
    
    model = build_model(optimizer = "momentum", momentum=mom)                      
    history = model.fit(X_train, y_train, epochs=100,
                    callbacks=[early_stopping, tensorboard_cb])
    
    mom_pkl.append((mom, history.history["mse"][-1], history.history["mae"][-1]))
save_to_pickle("mom", mom_pkl) 

Epoch 1/100
13/13 [==============================] - 0s 1ms/step - loss: 83728.2500 - mse: 83728.2500 - mae: 99.7617
Epoch 2/100
13/13 [==============================] - 0s 2ms/step - loss: 111.8574 - mse: 111.8574 - mae: 7.7539
Epoch 3/100
13/13 [==============================] - 0s 1ms/step - loss: 92.5026 - mse: 92.5026 - mae: 7.1335
Epoch 4/100
13/13 [==============================] - 0s 2ms/step - loss: 86.4758 - mse: 86.4758 - mae: 6.8017
Epoch 5/100
13/13 [==============================] - 0s 2ms/step - loss: 82.8550 - mse: 82.8550 - mae: 6.7412
Epoch 6/100
13/13 [==============================] - 0s 2ms/step - loss: 81.9884 - mse: 81.9884 - mae: 6.6559
Epoch 7/100
13/13 [==============================] - 0s 2ms/step - loss: 82.4515 - mse: 82.4515 - mae: 6.5522
Epoch 8/100
13/13 [==============================] - 0s 2ms/step - loss: 80.9828 - mse: 80.9828 - mae: 6.6153
Epoch 9/100
13/13 [==============================] - 0s 2ms/step - loss: 80.6428 - mse: 80.6428 - mae: 6.5149
E

## Autonomiczne przeszukiwanie przestrzeni hiperparametrów

In [12]:
#slownik zawierajacy przeszukiwanie wartosci parametrow
param_distribs = {
    "model__n_hidden": [0, 1, 2, 3],
    "model__n_neurons": [5, 25, 125],
    "model__learning_rate": [1e-6, 1e-5, 1e-4],
    "model__optimizer": ["sgd", "nesterov", "momentum", "adam"],
    "model__momentum": [0.1, 0.5, 0.9]
}

In [13]:
import scikeras
from scikeras.wrappers import KerasRegressor

es = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=1.0, verbose=1)

keras_reg = KerasRegressor(build_model, callbacks=[es])

### RandomizedSearchCV

In [14]:
from sklearn.model_selection import RandomizedSearchCV
rnd_search_cv = RandomizedSearchCV(keras_reg,
                                   param_distribs,
                                   n_iter=30,
                                   cv=3,
                                   verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_split=0.1)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Epoch 1/100
8/8 [==============================] - 1s 28ms/step - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 2/100
8/8 [==============================] - 0s 6ms/step - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 3/100
8/8 [==============================] - 0s 5ms/step - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 4/100
8/8 [==============================] - 0s 5ms/step - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 5/100
8/8 [==============================] - 0s 5ms/step - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 6/100
8/8 [==============================] - 0s 5ms/step - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 7/100
8/8 [==============================] - 0s 5ms/step - loss: nan - mse

/Users/kamilbernacik/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/kamilbernacik/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/kamilbernacik/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/Users/kamilbernacik/opt/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py", line 1100, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/Users/kamilbernacik/opt/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py", line 1697, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "/Users/kamilber

8/8 [==============================] - 0s 22ms/step - loss: 7657043506052988928.0000 - mse: 7657043506052988928.0000 - mae: 1007819584.0000 - val_loss: 2308499570688.0000 - val_mse: 2308499570688.0000 - val_mae: 1519374.6250
Epoch 2/100
8/8 [==============================] - 0s 5ms/step - loss: 2305448214528.0000 - mse: 2305448214528.0000 - mae: 1518370.1250 - val_loss: 2301122838528.0000 - val_mse: 2301122838528.0000 - val_mae: 1516945.1250
Epoch 3/100
8/8 [==============================] - 0s 5ms/step - loss: 2298081443840.0000 - mse: 2298081443840.0000 - mae: 1515942.2500 - val_loss: 2293769961472.0000 - val_mse: 2293769961472.0000 - val_mae: 1514519.7500
Epoch 4/100
8/8 [==============================] - 0s 8ms/step - loss: 2290738528256.0000 - mse: 2290738528256.0000 - mae: 1513518.5000 - val_loss: 2286441201664.0000 - val_mse: 2286441201664.0000 - val_mae: 1512098.2500
Epoch 5/100
8/8 [==============================] - 0s 5ms/step - loss: 2283419205632.0000 - mse: 2283419205632.0

/Users/kamilbernacik/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/kamilbernacik/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/kamilbernacik/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/Users/kamilbernacik/opt/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py", line 1100, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/Users/kamilbernacik/opt/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py", line 1697, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "/Users/kamilber

8/8 [==============================] - 0s 21ms/step - loss: 21785.2227 - mse: 21785.2227 - mae: 146.3550 - val_loss: 20512.6641 - val_mse: 20512.6641 - val_mae: 142.1184
Epoch 2/100
8/8 [==============================] - 0s 5ms/step - loss: 21782.3613 - mse: 21782.3613 - mae: 146.3454 - val_loss: 20509.8711 - val_mse: 20509.8711 - val_mae: 142.1087
Epoch 3/100
8/8 [==============================] - 0s 5ms/step - loss: 21779.5137 - mse: 21779.5137 - mae: 146.3358 - val_loss: 20507.0664 - val_mse: 20507.0664 - val_mae: 142.0990
Epoch 4/100
8/8 [==============================] - 0s 5ms/step - loss: 21776.6660 - mse: 21776.6660 - mae: 146.3262 - val_loss: 20504.2734 - val_mse: 20504.2734 - val_mae: 142.0893
Epoch 5/100
8/8 [==============================] - 0s 5ms/step - loss: 21773.8379 - mse: 21773.8379 - mae: 146.3167 - val_loss: 20501.4668 - val_mse: 20501.4668 - val_mae: 142.0796
Epoch 6/100
8/8 [==============================] - 0s 5ms/step - loss: 21770.9648 - mse: 21770.9648 - mae:

...

/Users/kamilbernacik/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/kamilbernacik/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/kamilbernacik/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/Users/kamilbernacik/opt/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py", line 1100, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/Users/kamilbernacik/opt/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py", line 1697, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "/Users/kamilber

8/8 [==============================] - 0s 21ms/step - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 2/100
8/8 [==============================] - 0s 5ms/step - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 3/100
8/8 [==============================] - 0s 9ms/step - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 4/100
8/8 [==============================] - 0s 6ms/step - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 5/100
8/8 [==============================] - 0s 6ms/step - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 6/100
8/8 [==============================] - 0s 5ms/step - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 7/100
8/8 [==============================] - 0s 6ms/step - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 8/10

/Users/kamilbernacik/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/kamilbernacik/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/kamilbernacik/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/Users/kamilbernacik/opt/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py", line 1100, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/Users/kamilbernacik/opt/anaconda3/lib/python3.9/site-packages/scikeras/wrappers.py", line 1697, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "/Users/kamilber

12/12 [==============================] - 1s 15ms/step - loss: 1156.2904 - mse: 1156.2904 - mae: 31.1206 - val_loss: 191.0138 - val_mse: 191.0138 - val_mae: 12.3603
Epoch 2/100
12/12 [==============================] - 0s 4ms/step - loss: 157.8184 - mse: 157.8184 - mae: 9.8982 - val_loss: 131.9712 - val_mse: 131.9712 - val_mae: 8.8789
Epoch 3/100
12/12 [==============================] - 0s 5ms/step - loss: 141.0938 - mse: 141.0938 - mae: 9.7339 - val_loss: 50.9592 - val_mse: 50.9592 - val_mae: 6.0747
Epoch 4/100
12/12 [==============================] - 0s 5ms/step - loss: 76.8893 - mse: 76.8893 - mae: 6.3340 - val_loss: 26.7590 - val_mse: 26.7590 - val_mae: 4.0263
Epoch 5/100
12/12 [==============================] - 0s 4ms/step - loss: 75.2245 - mse: 75.2245 - mae: 5.9670 - val_loss: 25.7716 - val_mse: 25.7716 - val_mae: 4.1436
Epoch 6/100
12/12 [==============================] - 0s 4ms/step - loss: 71.2877 - mse: 71.2877 - mae: 6.1335 - val_loss: 29.7247 - val_mse: 29.7247 - val_mae: 4.

RandomizedSearchCV(cv=3,
                   estimator=KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7f8ea1a3d6d0>], model=<function build_model at 0x7f8eab1efca0>),
                   n_iter=30,
                   param_distributions={'model__learning_rate': [1e-06, 1e-05,
                                                                 0.0001],
                                        'model__momentum': [0.1, 0.5, 0.9],
                                        'model__n_hidden': [0, 1, 2, 3],
                                        'model__n_neurons': [5, 25, 125],
                                        'model__optimizer': ['sgd', 'nesterov',
                                                             'momentum',
                                                             'adam']},
                   verbose=2)

### Best params i zapis

In [15]:
print(rnd_search_cv.best_params_)

{'model__optimizer': 'adam', 'model__n_neurons': 125, 'model__n_hidden': 3, 'model__momentum': 0.1, 'model__learning_rate': 0.0001}


In [16]:
import pickle
with open('rnd_search.pkl','wb') as f: pickle.dump(rnd_search_cv.best_params_, f)

#sprawdzenie zawartosci pliku
with open('rnd_search.pkl','rb') as f: print(pickle.load(f))

{'model__optimizer': 'adam', 'model__n_neurons': 125, 'model__n_hidden': 3, 'model__momentum': 0.1, 'model__learning_rate': 0.0001}
